# Zuiderzeeland

This script adds a new column "peilgebied_cat" and makes sure the peilgebieden allign with the HWS layer:
- peilgebied_cat = 0 -> peilgebied
- peigelbied_cat = 1 -> RHWS (boezem)
- peilgebied_cat = 2 -> NHWS 

In [81]:
import geopandas as gpd
import numpy as np
from general_functions import *

%load_ext autoreload
%autoreload 2

remove_cat_2 = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Zuiderzeeland

In [82]:
#define relative paths
waterschap = 'Zuiderzeeland'
waterschap2 = 'Zuiderzeeland'
data_path = f"/DATAFOLDER/projects/4750_20/Data_postprocessed/Waterschappen/{waterschap}/{waterschap}.gpkg"

# Waterschaps boundaries
grens_path = "/DATAFOLDER/projects/4750_30/Data_overig/Waterschapsgrenzen/Waterschapsgrenzen.geojson"
# Hoofdwatersysteem boundaries
hws_path = "/DATAFOLDER/projects/4750_30/Data_overig/HWS/krw_basins_vlakken.gpkg"
# Buffer boundaries
buffer_path =  "/DATAFOLDER/projects/4750_30/Data_overig/HWS/hws_buffer_zuiderzeeland.gpkg"
# Output folder
output_folder = f"/DATAFOLDER/projects/4750_30/Data_postprocessed/Waterschappen/{waterschap}"

### Load Files

In [83]:
# Load HHNK files
Zuiderzeeland = read_gpkg_layers(gpkg_path = data_path, 
                    variables = ['stuw', 
                                 'gemaal', 
                                 'hydroobject',
                                 'duikersifonhevel',
                                 'peilgebied', 
                                 'streefpeil',
                                 'aggregation_area',
                                ])
Zuiderzeeland['peilgebied'] = Zuiderzeeland['peilgebied'].to_crs('EPSG:28992')

# Load waterschap boundaries
gdf_grens = gpd.read_file(grens_path)
gdf_grens = gdf_grens.to_crs('EPSG:28992')
gdf_grens = gdf_grens.set_index('waterschap')

# Load hws
gdf_hws = gpd.read_file(hws_path)

# Load buffer
gdf_buffer = gpd.read_file(buffer_path)

In [84]:
Zuiderzeeland['peilgebied'].globalid.is_unique

True

## Select waterschap boundaries and clip hws layer

In [85]:
# Select boundaries HH Amstel, Gooi en Vecht
gdf_grens = gdf_grens.loc[['Waterschap Zuiderzeeland']]

# Use waterschap boudnaries to clip HWS layer
gdf_hws = gpd.overlay(gdf_grens, gdf_hws, how='intersection')

In [86]:
Zuiderzeeland['peilgebied']

,code,globalid,nen3610id,geometry
0,OR 56,dummy_globalid_peilgebied_0,dummy_nen3610id_peilgebied_0,"POLYGON Z ((167444.972 487009.101 0.000, 16762..."
1,OR 13,dummy_globalid_peilgebied_1,dummy_nen3610id_peilgebied_1,"POLYGON Z ((182383.808 509903.577 0.000, 18241..."
2,HT 6,dummy_globalid_peilgebied_2,dummy_nen3610id_peilgebied_2,"POLYGON Z ((158728.408 504422.232 0.000, 15873..."
3,SCHOTERPAD-D-TOCHT,dummy_globalid_peilgebied_3,dummy_nen3610id_peilgebied_3,"POLYGON Z ((179970.587 533631.593 0.000, 17996..."
4,ZWARTEMEERTOCHT,dummy_globalid_peilgebied_4,dummy_nen3610id_peilgebied_4,"POLYGON Z ((192476.653 518448.383 0.000, 19250..."
...,...,...,...,...
290,TA.02,dummy_globalid_peilgebied_290,dummy_nen3610id_peilgebied_290,"POLYGON Z ((189642.837 527330.192 0.000, 18985..."
291,URK 2,dummy_globalid_peilgebied_291,dummy_nen3610id_peilgebied_291,"POLYGON Z ((169418.497 519698.470 0.000, 16937..."
292,TA.13,dummy_globalid_peilgebied_292,dummy_nen3610id_peilgebied_292,"POLYGON Z ((187294.201 529202.794 0.000, 18747..."
293,3.11,dummy_globalid_peilgebied_293,dummy_nen3610id_peilgebied_293,"POLYGON Z ((160519.614 477230.775 0.000, 16016..."


## Peilgebied and HWS layer overlap:
1. Identify the overlapping areas
2. Clip
3. Calculate overlapping area percentage
4. Filter

In [87]:
# Step 1: Identify the Overlapping Areas and clip
overlaps = gpd.overlay(Zuiderzeeland['peilgebied'], gdf_hws, how='intersection', keep_geom_type=True)

# # Step 2: Subtract Overlapping Areas from the original polygons in each DataFrame
non_overlapping_peilgebied = gpd.overlay(Zuiderzeeland['peilgebied'], overlaps, how='difference', keep_geom_type=True)
overlaps = gpd.overlay(non_overlapping_peilgebied, gdf_hws, how='intersection', keep_geom_type=False)
 
# Step 3: Calculate Area Percentages
# Calculate the area of overlaps
overlaps['overlap_area'] = overlaps.area

# Step 4: Filter based on area Area Percentages
minimum_area = 100
print(f'Number of overlapping shapes without filter: {len(overlaps)}')
overlap_ids = overlaps.loc[overlaps['overlap_area'] > minimum_area]
overlap_ids = overlap_ids.globalid.to_list()
print(f'Number of overlapping shapes with filter: {len(overlap_ids)}')

Number of overlapping shapes without filter: 23
Number of overlapping shapes with filter: 0


In [88]:
Zuiderzeeland['peilgebied']

,code,globalid,nen3610id,geometry
0,OR 56,dummy_globalid_peilgebied_0,dummy_nen3610id_peilgebied_0,"POLYGON Z ((167444.972 487009.101 0.000, 16762..."
1,OR 13,dummy_globalid_peilgebied_1,dummy_nen3610id_peilgebied_1,"POLYGON Z ((182383.808 509903.577 0.000, 18241..."
2,HT 6,dummy_globalid_peilgebied_2,dummy_nen3610id_peilgebied_2,"POLYGON Z ((158728.408 504422.232 0.000, 15873..."
3,SCHOTERPAD-D-TOCHT,dummy_globalid_peilgebied_3,dummy_nen3610id_peilgebied_3,"POLYGON Z ((179970.587 533631.593 0.000, 17996..."
4,ZWARTEMEERTOCHT,dummy_globalid_peilgebied_4,dummy_nen3610id_peilgebied_4,"POLYGON Z ((192476.653 518448.383 0.000, 19250..."
...,...,...,...,...
290,TA.02,dummy_globalid_peilgebied_290,dummy_nen3610id_peilgebied_290,"POLYGON Z ((189642.837 527330.192 0.000, 18985..."
291,URK 2,dummy_globalid_peilgebied_291,dummy_nen3610id_peilgebied_291,"POLYGON Z ((169418.497 519698.470 0.000, 16937..."
292,TA.13,dummy_globalid_peilgebied_292,dummy_nen3610id_peilgebied_292,"POLYGON Z ((187294.201 529202.794 0.000, 18747..."
293,3.11,dummy_globalid_peilgebied_293,dummy_nen3610id_peilgebied_293,"POLYGON Z ((160519.614 477230.775 0.000, 16016..."


In [89]:
# Add occurence to geodataframe
peilgebieden_cat = []

for index, row in Zuiderzeeland['peilgebied'].iterrows():
       
    if 'LVA.01' in row.code:
        peilgebieden_cat.append(1)
        print('yes')
    elif '3.01' in row.code:
        peilgebieden_cat.append(1)
        print('yes')
    elif  'LAGE AFDELING' in row.code:
        print('yes')
        peilgebieden_cat.append(1)

    elif  'HOGE AFDELING' in row.code:
        print('yes')
        peilgebieden_cat.append(1)

    else:
        peilgebieden_cat.append(0)
        

        




# Add new column and drop old HWS_BZM column
Zuiderzeeland['peilgebied']['peilgebied_cat'] = peilgebieden_cat
# Zuiderzeeland['peilgebied'] = Zuiderzeeland['peilgebied'].drop(columns=['HWS_BZM'])

yes
yes
yes
yes
yes
yes


## Add nhws to ['peilgebied']

In [90]:
# update peilgebied dict key
gdf_hws['globalid'] = 'dummy_globalid_nhws_' + gdf_hws.index.astype(str)
gdf_hws['code'] = 'dummy_code_nhws_' + gdf_hws.index.astype(str)
gdf_hws['nen3610id'] = 'dummy_nen3610id_nhws_' + gdf_hws.index.astype(str)
gdf_hws['peilgebied_cat'] = 2

gdf_hws = gdf_hws[['globalid', 'code', 'nen3610id', 'peilgebied_cat', 'geometry']]

Zuiderzeeland['peilgebied'] = pd.concat([gdf_hws, Zuiderzeeland['peilgebied']])

## Add HWS to ['streefpeil']

In [91]:
# Create boezem streefpeil layer
streefpeil_hws = pd.DataFrame()
streefpeil_hws['waterhoogte'] = [np.nan] * len(gdf_hws)
streefpeil_hws['globalid'] = 'dummy_globalid_nhws_' + gdf_hws.index.astype(str)
streefpeil_hws['geometry'] = [None]* len(gdf_hws)

Zuiderzeeland['streefpeil'] = pd.concat([streefpeil_hws, Zuiderzeeland['streefpeil']])
Zuiderzeeland['streefpeil'] = gpd.GeoDataFrame(Zuiderzeeland['streefpeil'])

### Create buffer layer that ensures spatial match between peilgebied and hws layers based on the buffer layer

In [92]:
# buffer_polygon = gdf_buffer.geometry.iat[0].intersection(gdf_grens.geometry.iat[0])
# buffer_polygon = buffer_polygon.difference(shapely.geometry.MultiPolygon(gdf_hws.geometry.tolist()))
# buffer_polygon = buffer_polygon.difference(shapely.ops.unary_union(Zuiderzeeland['peilgebied'].geometry.tolist()))

# buffer_polygon = gpd.GeoDataFrame(buffer_polygon)
# buffer_polygon = buffer_polygon.set_geometry(0)
# buffer_polygon = buffer_polygon.dissolve()
# buffer_polygon = buffer_polygon.rename(columns={0:'geometry'})
# buffer_polygon = buffer_polygon.set_geometry('geometry')
# buffer_polygon = buffer_polygon.set_crs('EPSG:28992')

## Add buffer to ['peilgebied','streefpeil']

In [93]:
# # update peilgebied dict key
# buffer_polygon['globalid'] = 'dummy_globalid_nhws_buffer_' + buffer_polygon.index.astype(str)
# buffer_polygon['code'] = 'dummy_code_nhws_buffer_' + buffer_polygon.index.astype(str)
# buffer_polygon['nen3610id'] = 'dummy_nen3610id_nhws_buffer_' + buffer_polygon.index.astype(str)
# buffer_polygon['peilgebied_cat'] = 2

# buffer_polygon = buffer_polygon[['globalid', 'code', 'nen3610id', 'peilgebied_cat', 'geometry']]

# Zuiderzeeland['peilgebied'] = pd.concat([buffer_polygon, Zuiderzeeland['peilgebied']])

In [94]:
# # Create boezem streefpeil layer
# streefpeil_buffer = pd.DataFrame()
# streefpeil_buffer['waterhoogte'] = [np.nan]
# streefpeil_buffer['globalid'] = ['dummy_globalid_nhws_buffer_1']
# streefpeil_buffer['geometry'] = [None]


# Zuiderzeeland['streefpeil'] = pd.concat([streefpeil_buffer, Zuiderzeeland['streefpeil']])
# Zuiderzeeland['streefpeil'] = gpd.GeoDataFrame(Zuiderzeeland['streefpeil'])

In [95]:
if remove_cat_2:
    Zuiderzeeland['peilgebied'] = Zuiderzeeland['peilgebied'].loc[Zuiderzeeland['peilgebied'].peilgebied_cat !=2]

## Store output

In [96]:
Zuiderzeeland['peilgebied'].globalid.is_unique

True

In [97]:
for key in Zuiderzeeland.keys():   
    print(key)
    Zuiderzeeland[str(key)].to_file(f'{output_folder}/{waterschap2}.gpkg', layer = str(key), driver='GPKG')

stuw
gemaal
hydroobject
duikersifonhevel
peilgebied
streefpeil
aggregation_area
